Привет! В этом задании нужно реализовать метод опорных векторов с нуля и применить его для задачи бинарной классификации. Провести обучение на реальном датасете Breast Cancer, используя градиентный спуск, визуализировать процесс обучения и оценить качество предсказания.

Метод опорных векторов строит разделяющую гиперплоскость между классами, что можно эффективно использовать для задачи бинарной классификации. В реальных данных часто невозможно идеально разделить классы без ошибок. Поэтому используется подход с мягкими отступами (soft-margin SVM). В модели разрешено некоторое количество ошибок классификации, чтобы получить более устойчивую границу. Для этого вводится параметр $C$ — коэффициент штрафа за ошибку.

Чем больше значение $C$, тем сильнее модель пытается избегать ошибок (граница будет «жёстче»). Чем меньше значение $C$, тем больше модель позволяет себе допускать ошибки в пользу более широкой разделяющей области.

ЧТО НУЖНО СДЕЛАТЬ:

✅ 1. Загрузка и предобработка данных
* Использовать встроенный датасет Breast Cancer из sklearn.datasets.
* Перевести метки классов из {0,1} в {-1,+1}, чтобы соответствовать классической формулировке SVM.
* Проверить наличие пропусков и убедиться, что их нет. Для этого вывести сумму пропусков по каждому столбцу с помощью функции isnull(). Если сумма по всем столбцам равна 0 — продолжаем выполнение.
* Разделить данные на обучающую и тестовую выборки (80% / 20%) с использованием train_test_split.
* Стандартизировать признаки с помощью StandardScaler.

✅ 2. Обучение модели встроенной функцией

В библиотеке scikit-learn для задачи классификации используется готовая функция SVC(). Функция SVC() создаёт объект, который представляет собой готовую модель Support Vector Classifier. Этот объект позволяет:
* обучить модель на обучающих данных с помощью метода .fit(X, y);
* сделать предсказание на новых данных с помощью метода .predict(X).

Внутри этого объекта реализован поиск оптимальной разделяющей гиперплоскости с максимальным отступом между классами. Когда мы задаём параметр kernel='linear' (линейное ядро), модель будет строить простую линейную границу между классами (прямую в признаковом пространстве).

Что нужно сделать на этом шаге:
* Вызвать функцию SVC(kernel='linear',C=C0) для создания модели.
* Обучить модель на обучающих данных с помощью метода .fit.
* Построить предсказания на тестовых данных с помощью метода .predict.

✅ 3. Обучение собственной модели

Теперь создадим класс LinearSVM с конструктором init, принимающим параметры:
коэффициент штрафа ($C$), скорость обучения и количество итераций обучения (например, скорость обучения можно принять равной по умолчанию 0.001, а количество итераций — 1000). Внутри будет использоваться стохастический градиентный спуск вместо сложной оптимизации.

Итак, минимизируем следующую функцию: $L(w, b) = \frac{1}{2} \| w \|^2 + C \sum_{i=1}^{n} \max(0, 1 - y_i (w^T x_i + b))$ (используем hinge-loss функцию для формулировки задачи).

В методе fit для обучения модели на обучающих данных нужно проверять условие $y_i (w^T x_i + b) \geq 1$ (которое обеспечивает достаточный отступ от гиперплоскости) и в зависимости от его выполнения/невыполнения осуществлять градиентный спуск в нужном направлении.

В методе predict для предсказания на тестовых данных просто возвращаем знак $w^T x_i + b$.

После этого создадим объект нашего класса, запустив методы fit и predict.

✅ 4. Оценка результатов с помощью метрик
* accuracy (достоверность) — это доля правильных ответов: сколько раз модель вообще угадала (и позитивы, и негативы), делённая на общее число примеров.
* precision (точность) — это доля правильных среди предсказанных положительных: из всех объектов, которые модель назвала положительными, сколько реально оказались правильными (если модель сказала "это болезнь", precision показывает, как часто она не ошибалась)
* recall (полнота) — это доля найденных настоящих положительных: какую долю нашла модель из всех реальных положительных объектов (если на самом деле было 100 больных, recall показывает, сколько из них модель обнаружила)
* F1 — это среднее значение между precision и recall, которое наказывает за перекос в любую сторону (если precision высокий, а recall низкий — или наоборот — F1 упадёт, потому что важен баланс между ними)

Необходимо вывести значения данных метрик для обоих реализаций и сравнить их.

✅ 5. Визуализация результатов

* Используем PCA — это метод линейного уменьшения размерности, проецирующий данные на новое пространство из 2 главных компонент. Проще говоря, PCA позволяет «сжать» данные в 2 признака, которые лучше всего отражают структуру оригинальных данных.
* Трансформируем как обучающую, так и тестовую выборку.
* Модели, обученные на исходных 30 признаках, нельзя напрямую использовать на новых 2D-данных. Поэтому, нужно переобучить свою модель (LinearSVM) и встроенный SVC на новых признаках.
* Возьмём как границы области минимальные и максимальные значения по каждой из двух компонент, расширив диапазон на 1. Запустим функцию meshgrid, взяв по 200 точек по каждой оси. Эти точки — «псевдоданные», на которых мы будем проверять, что бы предсказала модель в каждой точке пространства.
* Запустим метод predict для каждого класса на матрице признаков размером $40000\times2$ (можно использовать функцию ravel) и с помощью reshape придадим форму сетки для удобства рисования.
* Построим области предсказаний с границей в виде полученной опорной гиперплоскости функцией plt.contourf (желательно поставить параметр $\alpha$ < 1 для красоты).
* И выведем 2D-проекцию реальных точек с помощью plt.scatter. Важно использовать опции: c=y — цвет зависит от реального класса точки, edgecolors='(цвет)' — обводка точек цветом для лучшей видимости.
* Построим графики для обеих реализаций.

✅ 6. Вопросы по итогу
* Проведите эксперимент, попробовав подобрать оптимальное значение $C_0$, исходя из метрик. Для сравнения приведите два других значения: одно — сильно меньше, другое — сильно больше. Как изменяется расположение тестовых точек относительно разделяющей границы? При маленьком $C$: становится ли больше точек, попавших "не в свою" область? При большом $C$: уменьшается ли количество ошибок, и становятся ли классы более чётко разделёнными? Сделайте вывод: как параметр регулирует компромисс между шириной разделяющей области и строгостью к ошибкам классификации.
* Как изменилась достоверность (accuracy) вашей реализации LinearSVM по сравнению со встроенной SVC? В какой модели итоговая достоверность оказалась выше? Какая модель справилась лучше на тестовых данных?
* При сравнении precision и recall у обеих моделей: какая модель лучше нашла положительные примеры (выше recall)? Какая модель допустила меньше пропущенных положительных случаев?
* Насколько чёткой получилась граница между классами на 2D-визуализации для каждой модели? Остались ли объекты, попавшие в «чужую» область? Каких ошибок было больше: ложных тревог или пропущенных положительных объектов?
* Есть ли на графиках после PCA объекты, которые трудно правильно классифицировать? Каких объектов больше: тех, что находятся далеко от своей области, или тех, что лежат на границе?